Going to play with different Neural Networks to try to model all of the data. I don't seem to have saved a varient of the dataframe where all of the stations are present

In [1]:
import polars as pl
from tqdm import tqdm
import os
import pickle
import re

from collections import defaultdict

In [2]:
# use CSVs in dir
# file_path = f"data/tfl_CSVs/"
directory = f"data/tfl_CSVs/"
downloaded_files = set(os.listdir(directory))
CSVs = downloaded_files
print(f"There are {len(CSVs)} files: {CSVs}")

There are 417 files: {'318JourneyDataExtract18May2022-24May2022.csv', '104JourneyDataExtract04Apr2018-10Apr2018.csv', '319JourneyDataExtract25May2022-31May2022.csv', '257JourneyDataExtract17Mar2021-23Mar2021.csv', '344JourneyDataExtract14Nov2022-20Nov2022.csv', '209JourneyDataExtract08Apr2020-14Apr2020.csv', '368JourneyDataExtract01May2023-07May2023.csv', '111JourneyDataExtract23May2018-29May2018.csv', '234JourneyDataExtract30Sep2020-06Oct2020.csv', '354JourneyDataExtract23Jan2023-29Jan2023.csv', '187JourneyDataExtract06Nov2019-12Nov2019.csv', '289JourneyDataExtract27Oct2021-02Nov2021.csv', '387JourneyDataExtract01Jan2024-14Jan2024.csv', '96JourneyDataExtract07Feb2018-13Feb2018.csv', '201JourneyDataExtract12Feb2020-18Feb2020.csv', '268JourneyDataExtract02Jun2021-08Jun2021.csv', '133JourneyDataExtract24Oct2018-30Oct2018.csv', '166JourneyDataExtract12Jun2019-18Jun2019.csv', '347JourneyDataExtract05Dec2022-11Dec2022.csv', '118JourneyDataExtract11July2018-17July2018.csv', '109JourneyDataEx

In [3]:
# Regular expression to find 4-digit years in the filenames
year_pattern = r'\d{4}'

# Set to store unique years
years = set()

year_files = defaultdict(list)

# Iterate through the file names and extract years
for file in CSVs:
    years = re.findall(year_pattern, file)
    for year in years:
        year_files[year].append(file)

year_files = dict(year_files)
print(year_files)

year_files = dict(sorted(year_files.items()))

for k, v in year_files.items():
    print(f"for year {k}, there are {len(v)} files: {v}")




{'2022': ['318JourneyDataExtract18May2022-24May2022.csv', '318JourneyDataExtract18May2022-24May2022.csv', '319JourneyDataExtract25May2022-31May2022.csv', '319JourneyDataExtract25May2022-31May2022.csv', '344JourneyDataExtract14Nov2022-20Nov2022.csv', '344JourneyDataExtract14Nov2022-20Nov2022.csv', '347JourneyDataExtract05Dec2022-11Dec2022.csv', '347JourneyDataExtract05Dec2022-11Dec2022.csv', '331JourneyDataExtract17Aug2022-23Aug2022.csv', '331JourneyDataExtract17Aug2022-23Aug2022.csv', '345JourneyDataExtract21Nov2022-27Nov2022.csv', '345JourneyDataExtract21Nov2022-27Nov2022.csv', '308JourneyDataExtract09Mar2022-15Mar2022.csv', '308JourneyDataExtract09Mar2022-15Mar2022.csv', '349JourneyDataExtract19Dec2022-25Dec2022.csv', '349JourneyDataExtract19Dec2022-25Dec2022.csv', '340JourneyDataExtract17Oct2022-23Oct2022.csv', '340JourneyDataExtract17Oct2022-23Oct2022.csv', '325JourneyDataExtract06Jul2022-12Jul2022.csv', '325JourneyDataExtract06Jul2022-12Jul2022.csv', '323JourneyDataExtract22Jun202

In [4]:
# CSVs = CSVs[:50]

In [5]:
# Initialize an empty Polars DataFrame
pickle_path = "pickle/CSV_to_df/"
all_cols = {}

CSVs_to_df = False

if CSVs_to_df:
    for k, v in year_files.items():
        merged_df = pl.DataFrame()
        cols = []

        progress_bar = tqdm(v, desc=f"Processing files from {k}")

        for idx, csv in enumerate(progress_bar):
            file_path = f"{directory}{csv}"
            # print(f"CSV: {csv}\n file_path: {file_path}")
            df = pl.read_csv(file_path, skip_rows_after_header=1, ignore_errors=True)
            for c in df.columns:
                if c in all_cols:
                    all_cols[c] += 1
                else:
                    all_cols[c] = 1

            # Rename columns and keep only 'Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name'
            renaming_dict = {
                'Number': 'Rental ID',
                'rental ID': 'Rental ID',
                'Rental Id': 'Rental ID',
                'Start date': 'Start Date',
                'StartStation Name': 'Start Station Name',
                # 'StartStation Id' : 'Start Station ID',
                # 'Start Station Id' : 'Start Station ID',
                # 'Start station number': 'Start Station ID',
                # 'StartStation Logical Terminal': 'Start Station ID',
                # 'EndStation Id' : 'End Station ID',
                # 'End Station Id' : 'End Station ID',
                # 'End station number': 'End Station ID',
                # 'EndStation Logical Terminal' : 'End Station ID',
                # 'endStationPriority_id' : 'End Station ID',
                'Start station': 'Start Station Name',
                'End date': 'End Date',
                'EndStation Name': 'End Station Name',
                'End station': 'End Station Name'
            }

            for old_col, new_col in renaming_dict.items():
                if old_col in df.columns:
                    df = df.rename({old_col: new_col})
            
            cols_to_keep = ['Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name']
            df = df.select(cols_to_keep)

            # Convert date columns
            df = df.with_columns([
                pl.col('Start Date').str.strptime(pl.Datetime, strict=False),
                pl.col('End Date').str.strptime(pl.Datetime, strict=False),
                pl.col('Start Station Name').cast(pl.Utf8),
                pl.col('End Station Name').cast(pl.Utf8)
            ])

            # Filter rows (if needed)
            # df = df.filter((pl.col("Start Station Name") == "Waterloo Station 3, Waterloo") |
            #                (pl.col("End Station Name") == "Waterloo Station 3, Waterloo"))
            
            # Concatenate DataFrames
            merged_df = pl.concat([merged_df, df])

            progress_bar.set_description(
                f"Creating merged_df for {k}. merged_df is {merged_df.estimated_size()/(1024*1024)}MB large"
            )
        pickle_dir = f"{pickle_path}/CSV_to_df_{k}"
        # save to pickle
        with open(pickle_dir, 'wb') as f:
            pickle.dump(merged_df, f)

In [6]:
# experiment to look at column values in the input CSVs
if CSVs_to_df:
    mapping_dict = {
                    'Number': 'Rental ID',
                    'rental ID': 'Rental ID',
                    'Rental Id': 'Rental ID',
                    'Start date': 'Start Date',
                    'StartStation Name': 'Start Station Name',
                    'StartStation Id' : 'Start Station ID',
                    'Start Station Id' : 'Start Station ID',
                    'Start station number': 'Start Station ID',
                    'StartStation Logical Terminal': 'Start Station ID',
                    'EndStation Id' : 'End Station ID',
                    'End Station Id' : 'End Station ID',
                    'End station number': 'End Station ID',
                    'EndStation Logical Terminal' : 'End Station ID',
                    'endStationPriority_id' : 'End Station ID',
                    'Start station': 'Start Station Name',
                    'End date': 'End Date',
                    'EndStation Name': 'End Station Name',
                    'End station': 'End Station Name'
                }

    def apply_dict_mapping(mapping, d):
        for k,v in mapping_dict.items():
            d[v] = d.get(k,0) + d.get(v,0)
            if k in d:
                del d[k]
        return d

    in_dict = all_cols

    out_dict = apply_dict_mapping(mapping_dict, in_dict)

    for c, n in out_dict.items():
        print(f"{c}: {n}")

In [7]:
#Generate Station to ID mapping
generate_station_to_ID_mapping = False
station_names_pickle_path = f"{pickle_path}/station_names"

if generate_station_to_ID_mapping:
    station_df = pl.Series(name="Station Names", values=[], dtype=pl.Utf8)
    for year in year_files.keys():
        #load from pickle
        pickle_dir = f"{pickle_path}/CSV_to_df_{year}"
        with open(pickle_dir, 'rb') as f:
            df = pickle.load(f)
            df_start = df['Start Station Name'].unique()
            df_end = df['End Station Name'].unique()
            station_df = pl.concat([station_df, df_start, df_end])
            
    station_df = station_df.drop_nans().drop_nulls()

    station_mapping = {s: i for i, s in enumerate(sorted(station_df.unique().to_list()))}

    mapping_df = pl.DataFrame({
        "Station Name": list(station_mapping.keys()),
        "ID": list(station_mapping.values())
    })

    
    
    
    with open(station_names_pickle_path, 'wb') as f:
            pickle.dump(mapping_df, f)
else:
     with open(station_names_pickle_path, 'rb') as f:
            mapping_df = pickle.load(f)

print(mapping_df.head())

shape: (5, 2)
┌─────────────────────────────────┬─────┐
│ Station Name                    ┆ ID  │
│ ---                             ┆ --- │
│ str                             ┆ i64 │
╞═════════════════════════════════╪═════╡
│ Abbey Orchard Street, Westmins… ┆ 0   │
│ Abbotsbury Road, Holland Park   ┆ 1   │
│ Aberdeen Place, St. John's Woo… ┆ 2   │
│ Aberfeldy Street, Poplar        ┆ 3   │
│ Abingdon Green, Westminster     ┆ 4   │
└─────────────────────────────────┴─────┘


In [8]:
df.head()

NameError: name 'df' is not defined

In [ ]:
# Generate yearly flows
generate_yearly_flows = True
flows_pickle_path = "pickle/flows"

if generate_yearly_flows:


    for year in tqdm(year_files.keys()):
        #load from pickle
        pickle_dir = f"{pickle_path}/CSV_to_df_{year}"
        with open(pickle_dir, 'rb') as f:
            df = pickle.load(f)

        df = df.join(mapping_df.rename({"Station Name": "Start Station Name", "ID": "Start ID"}), on="Start Station Name", how="left")
        df = df.join(mapping_df.rename({"Station Name": "End Station Name", "ID": "End ID"}), on="End Station Name", how="left")

        df_start = df["Rental ID", "Start Date", "Start ID"].rename({"Start Date": "Date", "Start ID": "ID"}).with_columns(pl.lit(-1).alias("flow"))
        df_end = df["Rental ID", "End Date", "End ID"].rename({"End Date": "Date", "End ID": "ID"}).with_columns(pl.lit(1).alias("flow"))
        df_flows = pl.concat([df_start, df_end]).sort("Date")

        with open(f"{flows_pickle_path}/flow_{year}", 'wb') as f:
            pickle.dump(mapping_df, f)


  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [01:17<00:00,  8.66s/it]


shape: (5, 4)
┌───────────┬─────────────────────┬─────┬──────┐
│ Rental ID ┆ Date                ┆ ID  ┆ flow │
│ ---       ┆ ---                 ┆ --- ┆ ---  │
│ i64       ┆ datetime[μs]        ┆ i64 ┆ i32  │
╞═══════════╪═════════════════════╪═════╪══════╡
│ 136450332 ┆ 2024-01-01 00:01:00 ┆ 899 ┆ -1   │
│ 136450333 ┆ 2024-01-01 00:01:00 ┆ 899 ┆ -1   │
│ 136450334 ┆ 2024-01-01 00:01:00 ┆ 899 ┆ -1   │
│ 136450332 ┆ 2024-01-01 00:01:00 ┆ 899 ┆ -1   │
│ 136450333 ┆ 2024-01-01 00:01:00 ┆ 899 ┆ -1   │
└───────────┴─────────────────────┴─────┴──────┘

In [10]:
df_flows.describe()

shape: (9, 5)
┌────────────┬──────────────┬────────────────────────────┬────────────┬────────────┐
│ statistic  ┆ Rental ID    ┆ Date                       ┆ ID         ┆ flow       │
│ ---        ┆ ---          ┆ ---                        ┆ ---        ┆ ---        │
│ str        ┆ f64          ┆ str                        ┆ f64        ┆ f64        │
╞════════════╪══════════════╪════════════════════════════╪════════════╪════════════╡
│ count      ┆ 9.818556e6   ┆ 9818556                    ┆ 9.818556e6 ┆ 9.818556e6 │
│ null_count ┆ 0.0          ┆ 0                          ┆ 0.0        ┆ 0.0        │
│ mean       ┆ 1.3771e8     ┆ 2024-03-04 21:50:28.210221 ┆ 445.901679 ┆ 0.0        │
│ std        ┆ 724225.08113 ┆ null                       ┆ 264.406969 ┆ 1.0        │
│ min        ┆ 1.36450332e8 ┆ 2024-01-01 00:01:00        ┆ 0.0        ┆ -1.0       │
│ 25%        ┆ 1.37078852e8 ┆ 2024-02-03 15:09:00        ┆ 219.0      ┆ -1.0       │
│ 50%        ┆ 1.37706639e8 ┆ 2024-03-06 12:34:00        ┆ 435.0      ┆ 1.0        │
│ 75%        ┆ 1.38331809e8 ┆ 2024-04-04 15:16:00        ┆ 677.0      ┆ 1.0        │
│ max        ┆ 1.38959725e8 ┆ 2024-05-13 12:43:00        ┆ 899.0      ┆ 1.0        │
└────────────┴──────────────┴────────────────────────────┴────────────┴────────────┘

In [15]:
def aggregate_by_time(df, time_str="15m", time_str_name='15minWindow'):
    df_agg = (
        df
        .with_columns(pl.col("Date").dt.truncate(time_str).alias(time_str_name))
        .group_by(["ID", time_str_name])
        .agg(pl.col("flow").sum().alias("total_flow"))
        )
    return df_agg
fifteen_min_agg = aggregate_by_time(df_flows)
print(fifteen_min_agg.describe())
print(f"merged_df is {fifteen_min_agg.estimated_size()/(1024*1024)}MB large")

shape: (9, 4)
┌────────────┬────────────┬────────────────────────────┬────────────┐
│ statistic  ┆ ID         ┆ 15minWindow                ┆ total_flow │
│ ---        ┆ ---        ┆ ---                        ┆ ---        │
│ str        ┆ f64        ┆ str                        ┆ f64        │
╞════════════╪════════════╪════════════════════════════╪════════════╡
│ count      ┆ 2.760397e6 ┆ 2760397                    ┆ 2.760397e6 │
│ null_count ┆ 0.0        ┆ 0                          ┆ 0.0        │
│ mean       ┆ 443.3166   ┆ 2024-03-03 11:13:17.285970 ┆ 0.0        │
│ std        ┆ 261.621068 ┆ null                       ┆ 3.289251   │
│ min        ┆ 0.0        ┆ 2024-01-01 00:00:00        ┆ -116.0     │
│ 25%        ┆ 218.0      ┆ 2024-02-02 09:00:00        ┆ -2.0       │
│ 50%        ┆ 434.0      ┆ 2024-03-04 11:45:00        ┆ 0.0        │
│ 75%        ┆ 671.0      ┆ 2024-04-03 00:15:00        ┆ 2.0        │
│ max        ┆ 899.0      ┆ 2024-05-13 12:30:00        ┆ 104.0      │
└─────